On va tenter de scrapp ADEME en utilisant les api(mais quel bordel sah)

je capte plus ah bah si c'est bon.
On va s'entrainer sur books to scrap pour etre sur de ce que l'on fait.
on va s'inspirer de fou du cours 5


In [29]:
from requests import get
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl


In [ ]:
url = "https://books.toscrape.com/"
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:146.0) Gecko/20100101 Firefox/146.0'
}
page = get(url=url, headers=headers)
page.raise_for_status() 
print("Success!")
page.ok

Success!


True

In [12]:
html_brut = page.text # 2. On initialise le parser
soup = BeautifulSoup(html_brut, 'html.parser')

In [25]:
soup
soup.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [18]:
soup.select('.product_pod')[0]

<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [19]:
produits = soup.select(".product_pod")

for produit in produits:
    titre = produit.select_one("h3 a")["title"]
    prix = produit.select_one(".price_color").text
    print(f"{titre} coûte {prix}")

A Light in the Attic coûte Â£51.77
Tipping the Velvet coûte Â£53.74
Soumission coûte Â£50.10
Sharp Objects coûte Â£47.82
Sapiens: A Brief History of Humankind coûte Â£54.23
The Requiem Red coûte Â£22.65
The Dirty Little Secrets of Getting Your Dream Job coûte Â£33.34
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull coûte Â£17.93
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics coûte Â£22.60
The Black Maria coûte Â£52.15
Starving Hearts (Triangular Trade Trilogy, #1) coûte Â£13.99
Shakespeare's Sonnets coûte Â£20.66
Set Me Free coûte Â£17.46
Scott Pilgrim's Precious Little Life (Scott Pilgrim #1) coûte Â£52.29
Rip it Up and Start Again coûte Â£35.02
Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991 coûte Â£57.25
Olio coûte Â£23.88
Mesaerion: The Best Science Fiction Stories 1800-1849 coûte Â£37.59
Libertarianism for Beginners coûte Â£51.33
It's Only the Himalayas coûte Â

Test de regex

insertion de mon regex dans mon test

In [23]:
for produit in produits:
    titre = produit.select_one("h3 a")["title"]

    prix = produit.select_one(".price_color").text

    prix_et = re.findall(r"[0-9.]+", prix)[0]    
    prix_nickel = float(prix_et)

    print(f"{titre} coûte {prix_nickel}")

A Light in the Attic coûte 51.77
Tipping the Velvet coûte 53.74
Soumission coûte 50.1
Sharp Objects coûte 47.82
Sapiens: A Brief History of Humankind coûte 54.23
The Requiem Red coûte 22.65
The Dirty Little Secrets of Getting Your Dream Job coûte 33.34
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull coûte 17.93
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics coûte 22.6
The Black Maria coûte 52.15
Starving Hearts (Triangular Trade Trilogy, #1) coûte 13.99
Shakespeare's Sonnets coûte 20.66
Set Me Free coûte 17.46
Scott Pilgrim's Precious Little Life (Scott Pilgrim #1) coûte 52.29
Rip it Up and Start Again coûte 35.02
Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991 coûte 57.25
Olio coûte 23.88
Mesaerion: The Best Science Fiction Stories 1800-1849 coûte 37.59
Libertarianism for Beginners coûte 51.33
It's Only the Himalayas coûte 45.17


Version sécurisée

In [24]:
for produit in produits:
    try:
        # 1. Sécurité sur le titre
        balise_titre = produit.select_one("h3 a")
        titre = balise_titre["title"] if balise_titre else "Titre inconnu"

        # 2. Sécurité sur la balise prix
        balise_prix = produit.select_one(".price_color")
        if balise_prix:
            texte_prix = balise_prix.text
            
            # 3. Sécurité sur l'extraction des chiffres
            chiffres = re.findall(r"[0-9.]+", texte_prix)
            
            if chiffres:
                prix_nickel = float(chiffres[0])
                print(f"{titre} coûte {prix_nickel}")
            else:
                print(f"{titre} : Aucun chiffre trouvé dans le prix")
        else:
            print(f"{titre} : Balise de prix manquante")

    except Exception as e:
        # En cas d'erreur imprévue, on affiche l'erreur et on passe au produit suivant
        print(f"Erreur sur un produit : {e}")
        continue

A Light in the Attic coûte 51.77
Tipping the Velvet coûte 53.74
Soumission coûte 50.1
Sharp Objects coûte 47.82
Sapiens: A Brief History of Humankind coûte 54.23
The Requiem Red coûte 22.65
The Dirty Little Secrets of Getting Your Dream Job coûte 33.34
The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull coûte 17.93
The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics coûte 22.6
The Black Maria coûte 52.15
Starving Hearts (Triangular Trade Trilogy, #1) coûte 13.99
Shakespeare's Sonnets coûte 20.66
Set Me Free coûte 17.46
Scott Pilgrim's Precious Little Life (Scott Pilgrim #1) coûte 52.29
Rip it Up and Start Again coûte 35.02
Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991 coûte 57.25
Olio coûte 23.88
Mesaerion: The Best Science Fiction Stories 1800-1849 coûte 37.59
Libertarianism for Beginners coûte 51.33
It's Only the Himalayas coûte 45.17


#Essayer de passer sur toutes les pages pour voir

In [27]:
# On définit l'URL de base avec un "placeholder" {} pour le numéro de page
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

# Liste pour stocker tous nos dictionnaires de produits
tous_les_produits = []

# On boucle de la page 1 à la page 50 (par exemple)
for numero_page in range(1, 6):
    url = base_url.format(numero_page)
    print(f"Extraction de la page {numero_page}...")
    
    try:
        reponse = requests.get(url)
        reponse.raise_for_status() # Système d'alarme 
        print("Success!")
        
        soup = BeautifulSoup(reponse.text, 'html.parser')
        produits = soup.select(".product_pod")

        # Si le parser ne trouve plus de produits, on arrête la boucle
        if not produits:
            break

        for produit in produits:
            # --- TA LOGIQUE SÉCURISÉE ---
            balise_titre = produit.select_one("h3 a")
            titre = balise_titre["title"] if balise_titre else "Inconnu"

            balise_prix = produit.select_one(".price_color")
            if balise_prix:
                chiffres = re.findall(r"[0-9.]+", balise_prix.text)
                if chiffres:
                    prix_nickel = float(chiffres[0])
                    
                    # On stocke dans notre liste au lieu de juste print
                    tous_les_produits.append({
                        "titre": titre,
                        "prix": prix_nickel,
                        "page": numero_page
                    })

        # Bonne pratique : petite pause pour ne pas surcharger le serveur
        time.sleep(0.5)

    except Exception as e:
        print(f"Erreur sur la page {numero_page} : {e}")
        break

print(f"Terminé ! {len(tous_les_produits)} produits récupérés.")

Extraction de la page 1...
Success!
Extraction de la page 2...
Success!
Extraction de la page 3...
Success!
Extraction de la page 4...
Success!
Extraction de la page 5...
Success!
Terminé ! 100 produits récupérés.


In [28]:
tous_les_produits

[{'titre': 'A Light in the Attic', 'prix': 51.77, 'page': 1},
 {'titre': 'Tipping the Velvet', 'prix': 53.74, 'page': 1},
 {'titre': 'Soumission', 'prix': 50.1, 'page': 1},
 {'titre': 'Sharp Objects', 'prix': 47.82, 'page': 1},
 {'titre': 'Sapiens: A Brief History of Humankind', 'prix': 54.23, 'page': 1},
 {'titre': 'The Requiem Red', 'prix': 22.65, 'page': 1},
 {'titre': 'The Dirty Little Secrets of Getting Your Dream Job',
  'prix': 33.34,
  'page': 1},
 {'titre': 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  'prix': 17.93,
  'page': 1},
 {'titre': 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  'prix': 22.6,
  'page': 1},
 {'titre': 'The Black Maria', 'prix': 52.15, 'page': 1},
 {'titre': 'Starving Hearts (Triangular Trade Trilogy, #1)',
  'prix': 13.99,
  'page': 1},
 {'titre': "Shakespeare's Sonnets", 'prix': 20.66, 'page': 1},
 {'titre': 'Set Me Free', 'prix': 17.46, 'page': 1},
 {'

mise sous tableau polars 

In [30]:
# 1. On transforme la liste en DataFrame Polars
df = pl.from_dicts(tous_les_produits)

# 2. On affiche les premières lignes pour vérifier
print(df.head())

# 3. Optionnel : On peut trier par prix croissant
df_trie = df.sort("prix")
print(df_trie)

shape: (5, 3)
┌─────────────────────────────────┬───────┬──────┐
│ titre                           ┆ prix  ┆ page │
│ ---                             ┆ ---   ┆ ---  │
│ str                             ┆ f64   ┆ i64  │
╞═════════════════════════════════╪═══════╪══════╡
│ A Light in the Attic            ┆ 51.77 ┆ 1    │
│ Tipping the Velvet              ┆ 53.74 ┆ 1    │
│ Soumission                      ┆ 50.1  ┆ 1    │
│ Sharp Objects                   ┆ 47.82 ┆ 1    │
│ Sapiens: A Brief History of Hu… ┆ 54.23 ┆ 1    │
└─────────────────────────────────┴───────┴──────┘
shape: (100, 3)
┌─────────────────────────────────┬───────┬──────┐
│ titre                           ┆ prix  ┆ page │
│ ---                             ┆ ---   ┆ ---  │
│ str                             ┆ f64   ┆ i64  │
╞═════════════════════════════════╪═══════╪══════╡
│ Patience                        ┆ 10.16 ┆ 5    │
│ In Her Wake                     ┆ 12.84 ┆ 2    │
│ Princess Between Worlds (Wide-… ┆ 13.34 ┆ 5    │
│

bloc final avec tout 

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl

# 1. CONFIGURATION
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:146.0) Gecko/20100101 Firefox/146.0'
}
tous_les_produits = []

print("🚀 Démarrage du scraping...")

# 2. BOUCLE SUR LES PAGES (ex: 1 à 5)
for numero_page in range(1, 6):
    url = base_url.format(numero_page)
    
    try:
        reponse = requests.get(url, headers=headers)
        reponse.raise_for_status() # Alerte en cas d'erreur 404 ou 500
        
        soup = BeautifulSoup(reponse.text, 'html.parser')
        produits = soup.select(".product_pod")

        if not produits:
            print("Fin des données trouvées.")
            break

        for produit in produits:
            try:
                # Extraction sécurisée du titre
                balise_titre = produit.select_one("h3 a")
                titre = balise_titre["title"] if balise_titre else "Inconnu"

                # Extraction sécurisée du prix
                balise_prix = produit.select_one(".price_color")
                if balise_prix:
                    # Nettoyage avec Regex
                    chiffres = re.findall(r"[0-9.]+", balise_prix.text)
                    if chiffres:
                        prix_nickel = float(chiffres[0])
                        
                        # Stockage dans la liste
                        tous_les_produits.append({
                            "titre": titre,
                            "prix": prix_nickel,
                            "page": numero_page
                        })
            except Exception as e:
                print(f"Erreur sur un produit de la page {numero_page}: {e}")

        print(f"✅ Page {numero_page} traitée.")
        time.sleep(0.5) # Pause pour être respectueux

    except Exception as e:
        print(f"❌ Erreur critique sur la page {numero_page}: {e}")
        break

# 3. TRAITEMENT DES DONNÉES AVEC POLARS
if tous_les_produits:
    df = pl.from_dicts(tous_les_produits)
    
    # Affichage de quelques statistiques rapides
    print("\n--- RÉSUMÉ DES DONNÉES ---")
    print(f"Nombre total de livres : {len(df)}")
    print(f"Prix moyen : {df['prix'].mean():.2f} £")
    
    # 4. EXPORTATION
    df.write_csv("resultats_scraping.csv")
    print("\n💾 Données sauvegardées dans 'resultats_scraping.csv'")
else:
    print("Aucune donnée n'a été récupérée.")